In [1]:
import pandas as pd

In [15]:
file_path = '/Users/henrychiu/Documents/fuel check project/fuel check data/combined_fuel_price_history_2022_2023_2024_cleaned.csv'
df = pd.read_csv(file_path)

In [16]:
# Convert PriceUpdatedDate to datetime format
df['PriceUpdatedDate'] = pd.to_datetime(df['PriceUpdatedDate'], dayfirst=True)

# Extract week number and year for grouping
df['Year'] = df['PriceUpdatedDate'].dt.year
df['Week'] = df['PriceUpdatedDate'].dt.strftime('%U').astype(int)



/var/folders/56/9ccg46j54r99nc2y8w_2gvx00000gn/T/ipykernel_13692/143786883.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['PriceUpdatedDate'] = pd.to_datetime(df['PriceUpdatedDate'], dayfirst=True)


In [21]:
df.head()

,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,PriceUpdatedDate,Price,Year,Week
0,BP CAMDEN,"12 ARGYLE ST, CAMDEN NSW 2570",CAMDEN,2570.0,BP,E10,2022-01-01 00:02:04,141.9,2022,0
1,BP CAMDEN,"12 ARGYLE ST, CAMDEN NSW 2570",CAMDEN,2570.0,BP,U91,2022-01-01 00:02:04,143.9,2022,0
2,Caltex Woolworths Woolooware,"449 Captain Cook Drive, Woolooware NSW 2230",Woolooware,2230.0,Caltex Woolworths,U91,2022-01-01 00:04:40,173.9,2022,0
3,Caltex Woolworths Woolooware,"449 Captain Cook Drive, Woolooware NSW 2230",Woolooware,2230.0,Caltex Woolworths,P95,2022-01-01 00:04:40,186.9,2022,0
4,Caltex Woolworths Woolooware,"449 Captain Cook Drive, Woolooware NSW 2230",Woolooware,2230.0,Caltex Woolworths,E10,2022-01-01 00:04:40,171.9,2022,0


In [22]:
# Step 1: Compute Weekly Average Price
weekly_avg = df.groupby([
    'ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand',
    'FuelCode', 'Year', 'Week'
])['Price'].mean().reset_index(name='WeeklyAvgPrice')

# Step 2: Compute Weekly Upper 5th Percentile
weekly_upper = df.groupby([
    'ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand',
    'FuelCode', 'Year', 'Week'
])['Price'].quantile(0.95).reset_index(name='WeeklyUpper5Percentile')

# Step 3: Compute Weekly Lower 5th Percentile
weekly_lower = df.groupby([
    'ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand',
    'FuelCode', 'Year', 'Week'
])['Price'].quantile(0.05).reset_index(name='WeeklyLower5Percentile')

# Merge all results
station_weekly_stats = weekly_avg.merge(weekly_upper, on=[
    'ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand',
    'FuelCode', 'Year', 'Week'
]).merge(weekly_lower, on=[
    'ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand',
    'FuelCode', 'Year', 'Week'
])

In [23]:
station_weekly_stats.head()

,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,Year,Week,WeeklyAvgPrice,WeeklyUpper5Percentile,WeeklyLower5Percentile
0,AMPOL WOOLWORTHS WAMBERAL,"656-658 THE ENTRANCE ROAD, WAMBERAL NSW 2260",WAMBERAL,2260.0,Ampol,E10,2022,18,184.9,185.8,184.0
1,AMPOL WOOLWORTHS WAMBERAL,"656-658 THE ENTRANCE ROAD, WAMBERAL NSW 2260",WAMBERAL,2260.0,Ampol,E10,2022,20,213.9,213.9,213.9
2,AMPOL WOOLWORTHS WAMBERAL,"656-658 THE ENTRANCE ROAD, WAMBERAL NSW 2260",WAMBERAL,2260.0,Ampol,E10,2022,21,211.9,212.8,211.0
3,AMPOL WOOLWORTHS WAMBERAL,"656-658 THE ENTRANCE ROAD, WAMBERAL NSW 2260",WAMBERAL,2260.0,Ampol,E10,2022,22,205.1,209.7,198.9
4,AMPOL WOOLWORTHS WAMBERAL,"656-658 THE ENTRANCE ROAD, WAMBERAL NSW 2260",WAMBERAL,2260.0,Ampol,E10,2022,23,197.9,197.9,197.9


In [25]:
# Save the output
optimized_output_path = '/Users/henrychiu/Documents/fuel check project/fuel check data/fuel_price_weekly_2022_2023_2024_v2.csv'
station_weekly_stats.to_csv(optimized_output_path, index=False)